# Assignment 4
### Due date: Monday, October 23, 2017

Name: Dong, Larry <br/>
Student ID: 260647775

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

%matplotlib inline

# 1
### a)

In [2]:
def backward_substitution(U, b):
    # U is upper-triangular
    x = np.zeros(len(b))
    
    for i in range(len(U), 0, -1):
        # n-1, n-2, ..., 0
        isolated = b[i-1]
        for j in range(len(U), i, -1):
            isolated -= U[(i-1), (j-1)]*x[(j-1)]
        
        x[(i-1)] = isolated/U[(i-1), (i-1)]
        
    return x

def forward_substitution(L, b):
    return np.flip(backward_substitution(np.flipud(np.fliplr(U)).T, np.flip(b, axis=0)), axis=0)

In [3]:
U = np.array([[-10, -7, 0],
             [0, 2, 6],
             [0, 0, 5]], dtype='float')
b = np.array([4, 8, -6])

x_back = backward_substitution(U, b)
x_forward = forward_substitution(U.T, b)

# verification
print(U@x_back)
print(U.T@x_forward)

[ 4.  8. -6.]
[ 4.  8. -6.]


### b)

In [52]:
def swap_rows(A, index_1, index_2):
    row1 = np.copy(A[index_1])
    row2 = np.copy(A[index_2])
    
    A[index_2] = row1
    A[index_1] = row2
    # return function not needed

def PLU_factorization(A):
    # A in a n x n matrix
    L = np.identity(len(A), dtype='float')
    U = np.copy(A)
    P = np.identity(len(A), dtype='float')
    
    for j in range(len(A)):
        for i in range(j+1, len(A)):
            
            if U[j, j] == 0:
                for k in range(i+1, len(A)):
                    temp_row_index = None
                    
                    if temp_row_index != None:
                        pass
                    
                    if (np.abs(U[k, j]) < 1e-5):
                        swap_rows(U, temp_row_index, i)
                        swap_rows(P, temp_row_index, i)
                # swap
            
            L[i, j] = U[i, j]/U[j, j]
            U[i] -= L[i, j]*U[j]
    
    # note: P.T = P
    return P, L, U 

In [53]:
A = np.array([[2, 4, -2], 
              [4, 9, -3], 
              [-2, -1, 7]], dtype='float')

P, L, U = PLU_factorization(A)

# verification
print(P.T@L@U)

[[ 2.  4. -2.]
 [ 4.  9. -3.]
 [-2. -1.  7.]]


In [54]:
y = forward_substitution(L, b)
x = backward_substitution(U, y)
L@U@x

array([ 2.,  4., -3.])

### c)

In [6]:
def solve(A, b):
    P, L, U = PLU_factorization(A)
    y = forward_substitution(L, P@b)
    x = backward_substitution(U, y)
    
    assert all(np.abs(A@x - b) < 1e-5)
    
    return x

# 2

In [12]:
LUD_iterative = (np.tril(A, k=-1), np.triu(A, k=1), np.diag(np.diag(A)))

def iterative_methods(A, b, x, iteration, method, tolerance=1e-5):
    # method 1 is Jacobi, 2 is Gauss-Seidel
    assert method in [1, 2]
    
    L, U, D = A
    
    if iteration == 0:
        if method == 1:
            print("Using the Jacobi method\n")
        elif method == 2:
            print("Using the Gauss-Seidel method\n")
    
    if method == 1:
        x_new = np.linalg.inv(D)@(b - (L+U)@x)
    elif method == 2:
        x_new = np.linalg.inv(L+D)@(b - U@x)
        
    print("Iteration {}: x = {}".format(iteration, x_new))
    
    if all(np.abs(x_new - x) < tolerance):
        return x_new
    else:
        return iterative_methods(A, b, x_new, iteration+1, method, tolerance)

In [13]:
iterative_methods(LUD_iterative, b, np.zeros_like(b)+1, 0, 1)

Using the Jacobi method

Iteration 0: x = [ 1.          0.77777778 -0.42857143]
Iteration 1: x = [ 0.01587302  0.3015873  -0.46031746]
Iteration 2: x = [ 0.93650794  0.72839506 -0.80952381]
Iteration 3: x = [-0.26631393  0.20282187 -0.48551272]
Iteration 4: x = [ 1.10884354  0.84541306 -0.904258  ]
Iteration 5: x = [-0.59508412  0.09465021 -0.41955712]
Iteration 6: x = [ 1.39114247  1.01351835 -1.01364543]
Iteration 7: x = [-1.04068213 -0.06727846 -0.31488525]
Iteration 8: x = [ 1.81967168  1.24645253 -1.16409182]
Iteration 9: x = [-1.65699688 -0.30788469 -0.15917202]
Iteration 10: x = [ 2.45659736  1.57227461 -1.37455406]
Iteration 11: x = [-2.51910328 -0.66111685  0.06935276]
Iteration 12: x = [ 3.39158645  2.03160793 -1.67133191]
Iteration 13: x = [-3.73454778 -1.17559351  0.40211155]
Iteration 14: x = [ 4.75329856  2.68272509 -2.09209844]
Iteration 15: x = [-5.45754861 -1.92105439  0.88418889]
Iteration 16: x = [ 6.72629768  3.60919568 -2.6908788 ]
Iteration 17: x = [-7.90927016 -2

RecursionError: maximum recursion depth exceeded

In [15]:
A@iterative_methods(LUD_iterative, b, np.zeros_like(b)+1, 0, 2)

Using the Gauss-Seidel method

Iteration 0: x = [ 1.          0.77777778 -0.46031746]
Iteration 1: x = [-0.01587302  0.74250441 -0.75560595]
Iteration 2: x = [-0.24061476  0.74396014 -0.81960991]
Iteration 3: x = [-0.30753018  0.75236567 -0.83752781]
Iteration 4: x = [-0.34225915  0.76182813 -0.8460986 ]
Iteration 5: x = [-0.36975485  0.77119151 -0.85261688]
Iteration 6: x = [-0.39499991  0.78023878 -0.85853729]
Iteration 7: x = [-0.41901484  0.78893861 -0.86415587]
Iteration 8: x = [-0.44203309  0.79729608 -0.86953859]
Iteration 9: x = [-0.46413075  0.80532303 -0.8747055 ]
Iteration 10: x = [-0.48535155  0.81303219 -0.87966727]
Iteration 11: x = [-0.50573166  0.82043609 -0.88443246]
Iteration 12: x = [-0.52530464  0.8275468  -0.88900893]
Iteration 13: x = [-0.54410252  0.83437592 -0.89340416]
Iteration 14: x = [-0.56215601  0.84093462 -0.89762534]
Iteration 15: x = [-0.57949457  0.84723359 -0.90167937]
Iteration 16: x = [-0.59614654  0.85328312 -0.90557285]
Iteration 17: x = [-0.61213

array([ 4.00001889,  8.0000069 , -6.        ])

# 4

In [40]:
def system(x1, x2, x3):
    return np.array([-np.cos(x1)/80-x1 + x2**2/9+ np.sin(x3)/3,
                     np.sin(x1)/3 - x2 + np.cos(x3)/3,
                    -np.cos(x1)/9 + x2**2/3 + np.sin(x3)/6 - x3])

def jacobian(x1, x2, x3):
    return np.array([[np.sin(x1)/80 - 1, 2*x2/9, np.cos(x3)/3],
                    [np.cos(x1)/3, -1, -np.sin(x3)/3],
                    [np.sin(x1)/9, 2*x2/3, np.cos(x3)/6-1]])

def nonlinear_newton_raphson(f, f_prime, x, tolerance=1e-5):
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    fx = f(x1, x2, x3)
    jacob = f_prime(x1, x2, x3)
    
    x_new = x - np.linalg.inv(jacob)@fx

    if all(np.abs(x_new - x) < tolerance):
        return x_new
    else:
        return nonlinear_newton_raphson(f, f_prime, x_new, tolerance)

In [41]:
solution_nonlinear = nonlinear_newton_raphson(matrix, matrix_prime, np.zeros_like(b)+1)

In [42]:
def phi_system(x_array):
    x1 = x_array[0]
    x2 = x_array[1]
    x3 = x_array[2]
    return np.array([-np.cos(x1)/80 + x2**2/9+ np.sin(x3)/3,
                     np.sin(x1)/3 + np.cos(x3)/3,
                    -np.cos(x1)/9 + x2**2/3 + np.sin(x3)/6]) 

phi_system(solution_nonlinear)

array([-0.03161529,  0.3213893 , -0.09192439])

In [44]:
test1 = np.identity(6)
swap_rows(test1, 2, 4)
print(test1)

[[ 1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.]]
